In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import os, glob, re, sys, random, unicodedata, collections
from tqdm import tqdm
from functools import reduce
import nltk
from collections import Counter

from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
%matplotlib inline

In [3]:
a=pd.read_json("meta-data.jsonl",lines=True)

In [4]:
a=a.dropna(axis=0)

In [5]:
c=pd.read_json("livivo_hq_100_candidates.jsonl",lines=True)

In [6]:
c

,qid:,qstr,candidates
0,1,covid-19 OR sars-cov-2,"[M33634079, M32328927, M32467201, M32333579, M..."
1,2,dementia,"[M32016328, M32016329, M30925608, M31476229, M..."
2,3,biodiversity,"[M31517113, M28569800, M32821644, M32295086, M..."
3,4,influenza,"[M32048225, M31534258, M30722953, M31478058, M..."
4,5,malaria,"[M30824502, M30550661, M29813596, M29813666, M..."
...,...,...,...
995,1446,epigenetik,"[M30643952, M29632998, M27299943, M29560499, M..."
996,1452,stroke AND unit,"[M33389780, M33394911, M33013663, M32115340, M..."
997,1453,praxis AND der AND intensivmedizin,"[M29556682, M22441683, M33053589, M32086542, M..."
998,1454,das AND gesundheitssystem AND in AND deutschland,"[M31594009, M31396676, M32023648, M31590198, M..."


In [7]:
z= a["ABSTRACT"][0]

In [8]:
a

,ABSTRACT,AUTHOR,DATABASE,DBRECORDID,DOI,EISSN,LANGUAGE,PISSN,PUBLCOUNTRY,PUBLDATE,PUBLYEAR,SOURCE,TITLE
0,[Objective!#!To assess the effect of mild seni...,"[Hunt, L, Morris, J C, Edwards, D, Wilson, B S]",[MEDLINE],M8315186,[10.1111/j.1532-5415.1993.tb07465.x],[1532-5415],[eng],[0002-8614],[United States],[1993-07],[1993],[Journal of the American Geriatrics Society],Driving performance in persons with mild senil...
3,[Objective!#!To determine the prevalence and r...,"[Ferreira, Mariana, Abbade, Luciana, Bocchi, S...",[MEDLINE],M32696899,[10.1590/0034-7167-2018-0475],[1984-0446],[eng],[0034-7167],[Brazil],[2020-07-13],[2020],[Revista brasileira de enfermagem],Incontinence-associated dermatitis in elderly ...
9,[Background!#!Australian guidelines advocate r...,"[Ng, Natalie Su Quin, Ayton, Darshini, Workman...",[MEDLINE],M32359111,[10.1111/imj.14869],[1445-5994],[eng],[1444-0903],[Australia],[2020-05-01],[2020],[Internal medicine journal],Understanding diagnostic settings and carer ex...
14,[Objective!#!To evaluate the results of cervic...,"[Lawson, H W, Lee, N C, Thames, S F, Henson, R...",[MEDLINE],M9794662,[10.1016/s0029-7844(98)00257-9],[1873-233X],[eng],[0029-7844],[United States],[1998-11],[1998],[Obstetrics and gynecology],Cervical cancer screening among low-income wom...
15,[The rapidly expanding field of mobile health ...,"[Biagianti, Bruno, Hidalgo-Mazzei, Diego, Meye...",[MEDLINE],M29025862,[10.1136/eb-2017-102765],[1468-960X],[eng],[1362-0347],[England],[2017-10-12],[2017],[Evidence-based mental health],Developing digital interventions for people li...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
239779,"[Psychostimulants, such as methylphenidate, ar...","[Ivanov, Iliyan, Liu, Xun, Clerkin, Suzanne, S...",[MEDLINE],M24491951,[10.1016/j.euroneuro.2014.01.017],[1873-7862],[eng],[0924-977X],[Netherlands],[2014-06],[2014],[European neuropsychopharmacology : the journa...,Methylphenidate and brain activity in a reward...
239782,[Purpose!#!This study aimed to define the acti...,"[Willis, Claire, Elliott, Catherine, Reid, Sio...",[MEDLINE],M33820452,[10.1080/09638288.2021.1907458],[1464-5165],[eng],[0963-8288],[England],[2021-04-05],[2021],[Disability and rehabilitation],"""Capturing the magic"": identifying the active ..."
239784,[Objective!#!This study explores the decision-...,"[Witt, Laurel B, Olsen, Delane, Ablah, Elizabeth]",[MEDLINE],M23345340,[10.1177/1524839912472504],[1552-6372],[eng],[1524-8399],[United States],[2013-11],[2013],[Health promotion practice],Motivating factors for small and midsized busi...
239785,[Survey response rates for physicians are fall...,"[Taylor, Tamara, Scott, Anthony]",[MEDLINE],M30384770,[10.1177/0163278718807744],[1552-3918],[eng],[0163-2787],[United States],[2018-11-01],[2018],[Evaluation & the health professions],Do Physicians Prefer to Complete Online or Mai...


## Pre Processing

In [9]:
#Datas are stored as a list in every cell, this operation applied to extract the list
for i in a.columns:
    a[i]=a[i].apply(lambda x: x[0] if isinstance(x, list) else x)

In [10]:
a["LANGUAGE"].value_counts()

eng    43634
ger     3794
por      142
fre      132
spa      129
jpn       84
rus       31
hun       28
kor       21
ita       13
nor       11
pol        8
chi        7
dut        5
ice        1
tur        1
Name: LANGUAGE, dtype: int64

In [11]:
def strip_accents(text):   
    nfkd = unicodedata.normalize('NFKD', text)
    newText = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    return re.sub('[^a-zA-Z0-9 \\\']', ' ', newText)

def tokenize_text(text):
    """Make all necessary preprocessing of text: strip accents and punctuation, remove the words only contains digit
    remove \n, tokenize our text, convert to lower case, remove stop words and 
    words with less than 2 chars.

    Parameters:
    text (str): Input text

    Returns:
    str: cleaned tokenized text

   """    
    WORD_MIN_LENGTH = 2
    STOP_WORDS = nltk.corpus.stopwords.words('english')
    text = strip_accents(text)
    text = re.sub(re.compile('\n'),' ',text)
    words = word_tokenize(text)
    words = [word.lower() for word in words]
    words = [word for word in words if word not in STOP_WORDS and len(word) >= WORD_MIN_LENGTH]
    words = [word for word in words if word.isdigit()==False]
    return words

In [12]:
a["ABSTRACT"]=a["ABSTRACT"].apply(lambda x: strip_accents(x))

In [13]:
tokenized=(a["ABSTRACT"].apply(lambda x: tokenize_text(x)))

## Building Inverted Index

In [14]:
def inverted_index(words):
    """Create a inverted index of words (tokens or terms) from a list of terms

    Parameters:
    words (list of str): tokenized document text

    Returns:
    Inverted index of document (dict)

   """       
    inverted = {}
    for index, word in enumerate(words):
        locations = inverted.setdefault(word, [])
        locations.append(index)
    return inverted

def inverted_index_add(inverted, doc_id, doc_index):
    """Insert document id into Inverted Index

    Parameters:
    inverted (dict): Inverted Index
    doc_id (int): Id of document been added
    doc_index (dict): Inverted Index of a specific document.

    Returns:
    Inverted index of document (dict)

   """        
    for word in doc_index.keys():
        locations = doc_index[word]
        indices = inverted.setdefault(word, {})
        indices[doc_id] = locations
    return inverted

In [15]:
inverted_index(tokenized[3])

{'objective': [0],
 'determine': [1],
 'prevalence': [2, 18, 45, 74],
 'risk': [3, 49, 65, 80],
 'factors': [4, 50, 81],
 'incontinence': [5, 43, 77],
 'associated': [6, 76],
 'dermatitis': [7, 17, 44, 75],
 'elderly': [8, 33],
 'method': [9],
 'cross': [10],
 'sectional': [11],
 'exploratory': [12],
 'study': [13, 71],
 'carried': [14],
 'public': [15],
 'hospitals': [16],
 'associations': [19],
 'obtained': [20],
 'calculating': [21],
 'ratio': [22, 27, 55, 58, 63, 69],
 'effect': [23],
 'dimension': [24],
 'estimated': [25],
 'odds': [26, 54, 57, 62, 68],
 'confidence': [28],
 'interval': [29],
 'statistical': [30],
 'significance': [31],
 'results': [32],
 'people': [34],
 'included': [35],
 'average': [36],
 'age': [37],
 'years': [38],
 'old': [39],
 'combined': [40],
 'fecal': [41],
 'urinary': [42],
 'pressure': [46, 66],
 'injuries': [47],
 'candidiasis': [48],
 'longer': [51],
 'hospital': [52],
 'stay': [53],
 'obesity': [56],
 'high': [59, 64, 73],
 'level': [60],
 'depende

In [16]:
inverted_doc_indexes = {}
files_with_index = []
files_with_tokens = {}
for i in tokenized.index:
    #Clean and Tokenize text of each document
    words = tokenized[i]
    #Store tokens
    files_with_tokens[i] = words

    doc_index = inverted_index(words)
    inverted_index_add(inverted_doc_indexes, i, doc_index)
    files_with_index.append(i)

## Running Boolean Search

In [17]:
def boolean_search(inverted, file_names, query):
    """Run a boolean search with AND operator between terms over 
    the inverted index.

    Parameters:
    inverted (dict): Inverted Index
    file_names (list): List with names of files (books)
    query (txt): Query text

    Returns:
    Names of books that matchs the query.

   """      
    # preprocess the user query using same function used to build Inverted Index
    words = [word for _, word in enumerate(tokenize_text(query)) if word in inverted]
    # list with a disctinct document match for each term from query
    results = [set(inverted[word].keys()) for word in words]
    # AND operator. Replace & for | to modify to OR behavior.
    docs = reduce(lambda x, y: x & y, results) if results else []
    return ([file_names[doc] for doc in docs])

In [40]:
files_with_index= pd.DataFrame(files_with_index)
files_with_index.index=files_with_index[0]

In [42]:
boolean_search(inverted_doc_indexes, files_with_index[0] , c["qstr"][15])

[86400, 220448, 108113, 166865, 243, 13876, 81043, 18131, 222108, 69918]

## Ranking with tf-idf

In [19]:
DF = {}
for word in inverted_doc_indexes.keys():
    DF[word] = len ([doc for doc in inverted_doc_indexes[word]])

total_vocab_size = len(DF)
print(total_vocab_size)

107331


In [20]:
tf_idf = {} # Our data structure to store Tf-Idf weights

N = len(files_with_tokens)

for doc_id in tokenized.index:
    tokens= tokenized[doc_id]
    counter = Counter(tokens)
    words_count = len(tokens)
    
    for token in np.unique(tokens):
        
        # Calculate Tf
        tf = counter[token] # Counter returns a tuple with each terms counts
        tf = 1+np.log(tf)
        
        # Calculate Idf
        if token in DF:
            df = DF[token]
        else:
            df = 0
        idf = np.log((N+1)/(df+1))
        
        # Calculate Tf-idf        
        tf_idf[doc_id, token] = tf*idf

## Ranked Search

In [21]:
def ranked_search(k, tf_idf_index, file_names, query):
    """Run ranked query search using tf-idf model.

    Parameters:
    k (int): number of results to return
    tf_idf_index (dict): Data Structure storing Tf-Idf weights to each 
                        pair of (term,doc_id) 
    file_names (list): List with names of files (books)
    query (txt): Query text

    Returns:
    Top-k names of books that matchs the query.

   """   
    tokens = tokenize_text(query)
    query_weights = {}
    for doc_id, token in tf_idf:
        if token in tokens:
            query_weights[doc_id] = query_weights.get(doc_id, 0) + tf_idf_index[doc_id, token]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)
    results = []
    for i in query_weights[:k]:
        results.append(file_names[i[0]])
    
    return results

In [113]:
ranked_search(100,tf_idf,files_with_index[0], c["qstr"][1])

[59966,
 166241,
 126200,
 140929,
 170427,
 202042,
 150442,
 180947,
 191597,
 204978,
 25344,
 39925,
 162556,
 226375,
 8408,
 88242,
 228531,
 4013,
 67276,
 77212,
 87038,
 108491,
 123889,
 130418,
 133635,
 199859,
 225052,
 233575,
 15547,
 71927,
 74974,
 103103,
 104675,
 110483,
 110807,
 166398,
 177125,
 183566,
 186841,
 189343,
 219836,
 230031,
 1677,
 7832,
 14767,
 29747,
 43516,
 61492,
 68329,
 79311,
 92688,
 112925,
 130111,
 132502,
 135293,
 147356,
 150382,
 155130,
 161201,
 168109,
 181554,
 208063,
 36836,
 51530,
 55435,
 60804,
 64930,
 88718,
 92659,
 111673,
 119579,
 121872,
 122148,
 126082,
 144077,
 145045,
 157622,
 167682,
 172923,
 179035,
 186298,
 187760,
 197497,
 202418,
 202993,
 204502,
 231432,
 239248,
 3694,
 6930,
 21076,
 23862,
 30561,
 31782,
 33256,
 36527,
 42043,
 48917,
 53009,
 57024]

In [123]:
searched= ranked_search(100,tf_idf,files_with_index[0], c["qstr"][1])

In [124]:
result=[]
for i in searched:
    result.append(a[a.index==i]["DBRECORDID"].values[0])

In [127]:
((pd.DataFrame(result))==(pd.DataFrame(c[c["qid:"]==1]["candidates"].values[0]))).sum()

0    0
dtype: int64

In [134]:
c

,qid:,qstr,candidates
0,1,covid-19 OR sars-cov-2,"[M33634079, M32328927, M32467201, M32333579, M..."
1,2,dementia,"[M32016328, M32016329, M30925608, M31476229, M..."
2,3,biodiversity,"[M31517113, M28569800, M32821644, M32295086, M..."
3,4,influenza,"[M32048225, M31534258, M30722953, M31478058, M..."
4,5,malaria,"[M30824502, M30550661, M29813596, M29813666, M..."
...,...,...,...
995,1446,epigenetik,"[M30643952, M29632998, M27299943, M29560499, M..."
996,1452,stroke AND unit,"[M33389780, M33394911, M33013663, M32115340, M..."
997,1453,praxis AND der AND intensivmedizin,"[M29556682, M22441683, M33053589, M32086542, M..."
998,1454,das AND gesundheitssystem AND in AND deutschland,"[M31594009, M31396676, M32023648, M31590198, M..."


In [110]:
a[a["DBRECORDID"]=="M33634079"]

,ABSTRACT,AUTHOR,DATABASE,DBRECORDID,DOI,EISSN,LANGUAGE,PISSN,PUBLCOUNTRY,PUBLDATE,PUBLYEAR,SOURCE,TITLE


In [133]:
a[a.index==166241]["ABSTRACT"].values

array(["Background   Little is known about staff's attitudes in Irish acute hospital settings towards people living with dementia and their perceived dementia knowledge  The aim of this study was to understand the general level of dementia knowledge and attitudes towards dementia in different types of hospital staff  as well as to explore the potential influence of previous dementia training and experience  having a family member with dementia  and the potential moderating effects of personal characteristics  This data was required to plan and deliver general and targeted educational interventions to raise awareness of dementia throughout the acute services     Methods   A cross sectional survey was carried out among a diverse range of hospital staff  n   1795  in three urban acute general hospitals in Ireland  including doctors  nurses  healthcare attendants  allied professionals  and general support staff  Participants' perceived dementia knowledge and attitudes were assessed as well

In [112]:
a

,ABSTRACT,AUTHOR,DATABASE,DBRECORDID,DOI,EISSN,LANGUAGE,PISSN,PUBLCOUNTRY,PUBLDATE,PUBLYEAR,SOURCE,TITLE
0,Objective To assess the effect of mild senil...,"Hunt, L",MEDLINE,M8315186,10.1111/j.1532-5415.1993.tb07465.x,1532-5415,eng,0002-8614,United States,1993-07,1993,Journal of the American Geriatrics Society,Driving performance in persons with mild senil...
3,Objective To determine the prevalence and ri...,"Ferreira, Mariana",MEDLINE,M32696899,10.1590/0034-7167-2018-0475,1984-0446,eng,0034-7167,Brazil,2020-07-13,2020,Revista brasileira de enfermagem,Incontinence-associated dermatitis in elderly ...
9,Background Australian guidelines advocate re...,"Ng, Natalie Su Quin",MEDLINE,M32359111,10.1111/imj.14869,1445-5994,eng,1444-0903,Australia,2020-05-01,2020,Internal medicine journal,Understanding diagnostic settings and carer ex...
14,Objective To evaluate the results of cervica...,"Lawson, H W",MEDLINE,M9794662,10.1016/s0029-7844(98)00257-9,1873-233X,eng,0029-7844,United States,1998-11,1998,Obstetrics and gynecology,Cervical cancer screening among low-income wom...
15,The rapidly expanding field of mobile health ...,"Biagianti, Bruno",MEDLINE,M29025862,10.1136/eb-2017-102765,1468-960X,eng,1362-0347,England,2017-10-12,2017,Evidence-based mental health,Developing digital interventions for people li...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
239779,Psychostimulants such as methylphenidate are...,"Ivanov, Iliyan",MEDLINE,M24491951,10.1016/j.euroneuro.2014.01.017,1873-7862,eng,0924-977X,Netherlands,2014-06,2014,European neuropsychopharmacology : the journal...,Methylphenidate and brain activity in a reward...
239782,Purpose This study aimed to define the activ...,"Willis, Claire",MEDLINE,M33820452,10.1080/09638288.2021.1907458,1464-5165,eng,0963-8288,England,2021-04-05,2021,Disability and rehabilitation,"""Capturing the magic"": identifying the active ..."
239784,Objective This study explores the decision m...,"Witt, Laurel B",MEDLINE,M23345340,10.1177/1524839912472504,1552-6372,eng,1524-8399,United States,2013-11,2013,Health promotion practice,Motivating factors for small and midsized busi...
239785,Survey response rates for physicians are falli...,"Taylor, Tamara",MEDLINE,M30384770,10.1177/0163278718807744,1552-3918,eng,0163-2787,United States,2018-11-01,2018,Evaluation & the health professions,Do Physicians Prefer to Complete Online or Mai...


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(a["ABSTRACT"])
vectorizer.get_feature_names_out()
print(X.shape)

(48041, 111639)


In [11]:
print(X)

  (0, 30967)	0.027894887718337983
  (0, 93588)	0.02433232049457829
  (0, 62494)	0.027224184622861487
  (0, 92061)	0.033706506565205914
  (0, 24248)	0.031781756886177716
  (0, 74681)	0.02157178249324583
  (0, 13983)	0.03424797038591601
  (0, 82596)	0.02482616900479205
  (0, 13725)	0.01549365226662926
  (0, 30874)	0.027233702143947984
  (0, 83544)	0.05420179133719774
  (0, 68966)	0.03453570205889703
  (0, 12787)	0.01392090174141077
  (0, 38339)	0.04574629661123993
  (0, 80925)	0.03415129349051424
  (0, 59169)	0.04373844011886216
  (0, 45262)	0.05795279741681794
  (0, 94356)	0.07207876176648798
  (0, 88615)	0.056628730952955925
  (0, 95247)	0.02721657886049925
  (0, 25210)	0.019327213264825924
  (0, 109329)	0.021800960390999106
  (0, 26393)	0.03490448040341968
  (0, 99943)	0.0376737837160723
  (0, 77659)	0.0901940748662238
  :	:
  (48040, 20277)	0.03722989652971473
  (48040, 86205)	0.04786294918965753
  (48040, 21461)	0.08135798198563943
  (48040, 34073)	0.018578655701843094
  (48040, 101